In [ ]:
from functools import cache

In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

In [ ]:
input_file = "input.txt"

In [ ]:
file = open(input_file, 'r')
for ix, line in enumerate(file):
    row, groups =line.split(' ')
    groups = [int(g) for g in groups.split(',')]
    qpp = '\n'.join([f"MATCH (head:Head WHERE head.row_id={ix})",
    "(()-->(:OK|UNKNOWN))*",
    "(()-->(:OK|UNKNOWN))+\n".join([f"(()-->(g_{jx}:NOK|UNKNOWN)){{{g}}}" for jx, g in enumerate(groups)]),
    "(()-->(:OK|UNKNOWN|Tail))*(:Tail)",
    "WITH DISTINCT head, "+', '.join([f"g_{jx}" for jx, _ in enumerate(groups)]),
    "WITH head, count(*) as matches",
    "SET head.matches = matches"])
    gds.run_cypher("""
    CREATE (:Line {id:$ix, row:$row, groups:$groups, qpp:$qpp})
    """, {'ix':ix, 'row':row, 'groups':groups, 'qpp':qpp})

In [ ]:
for id, row in gds.run_cypher("""MATCH (line:Line) RETURN line.id AS id, line.row AS row""").itertuples(index=False):
    gds.run_cypher("""
    WITH $row AS row
    UNWIND split(row, '') AS c
    CREATE (s:Spring {row_id:$id, state:c})
    WITH collect(s) AS springs
    CALL apoc.nodes.link(springs, 'NEXT')
    """, {'row':row, 'id':id})


In [ ]:
queries = """MATCH (s:Spring)
WHERE NOT EXISTS {()-[:NEXT]->(s)}
SET s:First;
MATCH (s:Spring)
WHERE NOT EXISTS {(s)-[:NEXT]->()}
SET s:Last;
MATCH (s:Spring {state:'.'})
SET s:OK;
MATCH (s:Spring {state:'#'})
SET s:NOK;
MATCH (s:Spring {state:'?'})
SET s:UNKNOWN;
MATCH (f:First)
MERGE (h:Head {row_id:f.row_id})
MERGE (h)-[:NEXT]->(f);
MATCH (l:Last)
MERGE (t:Tail {row_id:l.row_id})
MERGE (l)-[:NEXT]->(t)
""".split(';')
for query in queries:
    gds.run_cypher(query)

In [ ]:
gds.run_cypher("""CALL apoc.periodic.iterate('MATCH (l:Line) RETURN l.qpp AS qpp',
'CALL apoc.cypher.runWrite(qpp,{}) YIELD value RETURN value.count AS count',
{parallel:TRUE})""")

In [ ]:
gds.run_cypher("""MATCH (h:Head)
RETURN sum(h.matches) AS part1""")

# Part 1
No graph database

In [ ]:
@cache
def iter_cuts(chunk):
    res = []
    for ix, _ in enumerate(chunk):
        if not chunk[ix] == "#":
            res.append((chunk[0:ix], chunk[ix+1:]))
    return res

In [ ]:
@cache
def one_group_arrangement(chunk, group):
    res = set()
    for ix in range(0, 1+len(chunk) - group):
        if not ('#' in chunk[:ix]+chunk[ix+group:] or '.' in chunk[ix:ix+group]):
            #print("in if",ix)
            res.add(("."*ix) + ("#"*group) + ("."*(len(chunk)-(ix+group))))
    return res
    

In [ ]:
@cache
def arrangements(chunk, groups):
    res = set()
    if len(groups) == 1:
        for match in one_group_arrangement(chunk, groups[0]):
            res.add(match)
        return res
    first, *others = groups
    others = tuple(others)
    for h, t in iter_cuts(chunk):
        for start in one_group_arrangement(h, first):
            for end in arrangements(t, others):
                res.add(start+'.'+end)
    return res
    
       


In [ ]:
file = open(input_file, 'r')
part1 = 0
for ix, line in enumerate(file):
    row, groups =line.split(' ')
    groups = tuple([int(g) for g in groups.split(',')])
    arr = len(arrangements(row,groups))
    part1 += arr
part1

